In [3]:
import torch, os, platform, sys
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

PyTorch: 2.8.0+cu126
CUDA available: True
Wed Nov 12 21:26:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   45C    P8             16W /   72W |       3MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
!cd /content
!git clone https://github.com/Makiato1999/HRI-EMO.git
!cd HRI-EMO

Cloning into 'HRI-EMO'...
remote: Enumerating objects: 247, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 247 (delta 132), reused 167 (delta 56), pack-reused 0 (from 0)
Receiving objects: 100% (247/247), 154.74 KiB | 22.11 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [6]:
!cd /content

!cp /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/mosei_features_and_index.zip .
!unzip -q mosei_features_and_index.zip -d .

In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install tqdm pandas numpy

Looking in indexes: https://download.pytorch.org/whl/cu121


In [10]:
import os, sys

# 看看仓库是不是在 /content/HRI-EMO
print("ls /content ->")
print(os.listdir("/content"))

# 如果你看到 'HRI-EMO' 这个文件夹，就执行切换
os.chdir("/content/HRI-EMO")
print("✅ cwd:", os.getcwd())

# 确保可以 import 本项目的 models 包
sys.path.append(os.getcwd())

ls /content ->
['.config', 'HRI-EMO', 'drive', 'data', 'features', 'mosei_features_and_index.zip', 'sample_data']
✅ cwd: /content/HRI-EMO


In [9]:
!cd /content/HRI-EMO

# !python -m scripts.fusion.train_mosei_fusion_seq_level_decoder \
#   --index_csv ../data/mosei_index_splits.csv \
#   --audio_dir ../features/mosei/seq_level/audio \
#   --text_dir ../features/mosei/seq_level/text \
#   --epochs 25 \
#   --batch_size 8 \
#   --grad_accum 4 \
#   --warmup_ratio 0.1 \
#   --beta_entropy 1e-3 \
#   --max_len_audio 300 \
#   --max_len_text 128 \
#   --d_model 256 \
#   --n_heads 4 \
#   --num_layers_fusion 2 \
#   --num_layers_decoder 2 \
#   --dropout 0.2 \
#   --lr 1e-4 \
#   --weight_decay 1e-2 \
#   --out_dir /content/drive/MyDrive/HRI-EMO-results/mosei_fusion_decoder_small \
#   --seed 1234

!python -m scripts.fusion.train_mosei_fusion_seq_level_decoder \
  --index_csv ../data/mosei_index_splits.csv \
  --audio_dir ../features/mosei/seq_level/audio \
  --text_dir ../features/mosei/seq_level/text \
  --epochs 20 \
  --batch_size 8 \
  --grad_accum 4 \
  --warmup_ratio 0.1 \
  --beta_entropy 1e-3 \
  --max_len_audio 300 \
  --max_len_text 128 \
  --d_model 384 \
  --n_heads 6 \
  --num_layers_fusion 2 \
  --num_layers_decoder 2 \
  --dropout 0.2 \
  --lr 1e-4 \
  --weight_decay 1e-2 \
  --num_workers 2 \
  --select_by macro_auc \
  --save_calibrated_ths \
  --out_dir /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small \
  --seed 1234

# python -m scripts.fusion.train_mosei_fusion_seq_level_decoder \
#   --index_csv ../data/mosei_index_splits.csv \
#   --audio_dir ../features/mosei/seq_level/audio \
#   --text_dir ../features/mosei/seq_level/text \
#   --epochs 25 \
#   --batch_size 8 \
#   --grad_accum 4 \
#   --warmup_ratio 0.1 \
#   --beta_entropy 5e-4 \
#   --max_len_audio 300 \
#   --max_len_text 128 \
#   --d_model 384 \
#   --n_heads 6 \
#   --num_layers_fusion 3 \
#   --num_layers_decoder 3 \
#   --dropout 0.3 \
#   --lr 1e-4 \
#   --weight_decay 1e-2 \
#   --num_workers 2 \
#   --select_by macro_auc \
#   --save_calibrated_ths \
#   --out_dir /content/drive/MyDrive/HRI-EMO-results/mosei_fusion_decoder_384deeper \
#   --seed 1234


[Dataset] Final size: 16327 samples
[Dataset] Final size: 1871 samples
[pos_weight] {'emo_happy': np.float32(0.87), 'emo_sad': np.float32(2.82), 'emo_anger': np.float32(3.63), 'emo_fear': np.float32(8.94), 'emo_disgust': np.float32(4.53), 'emo_surprise': np.float32(11.27)}

=== Epoch 1/20 ===
[Val Calibrated] macro-F1=0.403 | thresholds=[0.05 0.05 0.1  0.1  0.05 0.1 ]
[Val Metrics] Loss=0.4239 | micro-F1=0.002 | macro-F1=0.003 | macro-AUC=0.645
Train Loss: 0.4915 | F1 micro/macro: 0.084/0.075 | AUC macro: 0.555 | Mean β: 0.495  ||  Val Loss: 0.4239 | F1 micro/macro: 0.002/0.003 | AUC macro: 0.645 | Mean β: 0.493 | Calib macro-F1: 0.403

=== Epoch 2/20 ===
[Val Calibrated] macro-F1=0.423 | thresholds=[0.05 0.15 0.2  0.15 0.25 0.2 ]
[Val Metrics] Loss=0.4156 | micro-F1=0.018 | macro-F1=0.024 | macro-AUC=0.684
Train Loss: 0.4310 | F1 micro/macro: 0.035/0.038 | AUC macro: 0.662 | Mean β: 0.490  ||  Val Loss: 0.4156 | F1 micro/macro: 0.018/0.024 | AUC macro: 0.684 | Mean β: 0.484 | Calib ma

In [24]:
# ⚠️ 删除重复目录
!rm -rf /content/HRI-EMO

# 重新克隆
%cd /content
!git clone https://github.com/Makiato1999/HRI-EMO.git
%cd HRI-EMO

# 确认路径正确
!pwd
!ls


/content
Cloning into 'HRI-EMO'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (262/262), done.
remote: Compressing objects: 100% (199/199), done.
remote: Total 262 (delta 140), reused 175 (delta 57), pack-reused 0 (from 0)
Receiving objects: 100% (262/262), 157.74 KiB | 22.53 MiB/s, done.
Resolving deltas: 100% (140/140), done.
/content/HRI-EMO
/content/HRI-EMO
models	notebooks  README.md  scripts  tests  tools
